In [1]:
import os
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf

In [2]:
# Global Variables
BATCH_SIZE = 16
IMG_SIZE = 224

In [3]:
# Paths: For local machine
train_dir = 'C:/Users/Nick/Documents/SchoolStuff/spring2024/machineLearning/final_project/tradingpatterns/stock_patterns/train'
validation_dir = 'C:/Users/Nick/Documents/SchoolStuff/spring2024/machineLearning/final_project/tradingpatterns/stock_patterns/validation'

# Create training and validation dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            color_mode='rgb',
                                                            image_size=(IMG_SIZE, IMG_SIZE))

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 color_mode='rgb',
                                                                 image_size=(IMG_SIZE, IMG_SIZE))
class_names = train_dataset.class_names

Found 6404 files belonging to 4 classes.
Found 5984 files belonging to 4 classes.


In [4]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [5]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (IMG_SIZE, IMG_SIZE) + (3,)
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [6]:
# This feature extractor converts each 255x255 image into a 5x5x1280 block of features
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

In [7]:
# Freeze the model
base_model.trainable = False

# Adding a classification head
global_max_layer = tf.keras.layers.GlobalMaxPooling2D()
feature_batch_max = global_max_layer(feature_batch)

In [8]:
# Add a dense layer to convert it to a single prediction per image
prediction_layer = tf.keras.layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_max)

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
# Add the preprocessing layer
x = preprocess_input(inputs)
# Add the base model after, keeping training to false
x = base_model(x, training=False)
# Add max pooling layer
x = global_max_layer(x)
# Add dropout layer
x = tf.keras.layers.Dropout(0.2)(x)
# Add softmax prediction layer
outputs = prediction_layer(x)
# Create a model from inputs, outputs.
model = tf.keras.Model(inputs, outputs)

In [9]:
# Compile the model
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
initial_epochs = 10

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    steps_per_epoch=10,
                    validation_data=validation_dataset,
                    validation_steps=10)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 596ms/step - accuracy: 0.1686 - loss: 3.5502 - val_accuracy: 0.3187 - val_loss: 2.2875
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 348ms/step - accuracy: 0.3398 - loss: 2.4112 - val_accuracy: 0.3375 - val_loss: 1.9466
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 360ms/step - accuracy: 0.1998 - loss: 2.5491 - val_accuracy: 0.3063 - val_loss: 1.9826
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 360ms/step - accuracy: 0.3857 - loss: 1.9392 - val_accuracy: 0.3063 - val_loss: 1.8747
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 354ms/step - accuracy: 0.3027 - loss: 2.1438 - val_accuracy: 0.3313 - val_loss: 1.7806
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 356ms/step - accuracy: 0.2495 - loss: 2.2821 - val_accuracy: 0.3438 - val_loss: 1.8649
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 356ms/step - accuracy: 0.2564 - loss: 2.1057 - val_accuracy: 0.3313 - val_loss: 1.7719
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 355ms/step - accuracy: 0.2885 - loss: 2.2559 - val_accuracy: 0

In [11]:
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# 
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# 
# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.ylabel('Accuracy')
# plt.ylim([0, .5])
# plt.title('Training and Validation Accuracy')
# 
# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.ylabel('Cross Entropy')
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.ylim([0, 10])
# plt.show()

In [12]:
base_model.trainable = True

In [13]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 180

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  311


In [14]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), # False when we have a Softmax layer
              metrics=['accuracy'])

In [15]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=len(history.epoch),
                         steps_per_epoch=10,
                         validation_data=validation_dataset,
                         validation_steps=10)

Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 777ms/step - accuracy: 0.3361 - loss: 2.5509 - val_accuracy: 0.3000 - val_loss: 1.9378
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 530ms/step - accuracy: 0.2815 - loss: 2.2660 - val_accuracy: 0.3000 - val_loss: 2.0929
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 527ms/step - accuracy: 0.3220 - loss: 2.1894 - val_accuracy: 0.2562 - val_loss: 2.3157
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 527ms/step - accuracy: 0.3131 - loss: 1.9229 - val_accuracy: 0.3375 - val_loss: 2.0359
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 525ms/step - accuracy: 0.2289 - loss: 2.1126 - val_accuracy: 0.2625 - val_loss: 2.1318
Epoch 16/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 529ms/step - accuracy: 0.2748 - loss: 2.1233 - val_accuracy: 0.2937 - val_loss: 2.0235
Epoch 17/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 529ms/step - accuracy: 0.2253 - loss: 2.3121 - val_accuracy: 0.2125 - val_loss: 2.1145
Epoch 18/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 521ms/step - accuracy: 0.2493 - loss: 2.0832 - val_acc

In [16]:
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

74/74 ━━━━━━━━━━━━━━━━━━━━ 13s 171ms/step - accuracy: 0.2771 - loss: 1.9763
Test accuracy : 0.28125


In [17]:
# acc += history_fine.history['accuracy']
# val_acc += history_fine.history['val_accuracy']
# 
# loss += history_fine.history['loss']
# val_loss += history_fine.history['val_loss']
# acc = history.history['accuracy']
# val_acc = history.history['val_accuracy']
# 
# loss = history.history['loss']
# val_loss = history.history['val_loss']
# 
# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.legend(loc='lower right')
# plt.ylabel('Accuracy')
# plt.ylim([0, 1])
# plt.title('Training and Validation Accuracy')
# 
# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend(loc='upper right')
# plt.ylabel('Cross Entropy')
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.ylim([0, 10])
# plt.show()

In [18]:
# Add a softmax layer to our model and apply our test data
# probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# image_batch, label_batch = test_dataset.as_numpy_iterator().next()
# predictions = probability_model.predict(image_batch)

In [19]:
# plt.figure(figsize=(10, 10))
# for i in range(9):
#   ax = plt.subplot(3, 3, i + 1)
#   plt.imshow(image_batch[i].astype("uint8"))
#   guess = str(class_names[np.argmax(predictions[i+9])])
#   actual = str(class_names[label_batch[i+9]])
#   title = "Prediction: " + guess + "\n" + "Actual: " + actual
#   plt.title(title)
#   plt.axis("off")